In [ ]:
from load_feat_pd import *


if __name__ == '__main__':
    
    base_folder_path = Path('/data/storage025/Turntaking/wavs_single_channel_normalized_nosil') if norm else Path('/data/storage025/Turntaking/wavs_single_channel_nosil')

    feats2level = {
        'jitter': 'utt',
        'shimmer': 'utt',
        'rp': 'utt',
        'f0': 'frame',
        'energy': 'frame'
    }

    np.set_printoptions(precision=2)
    allfeats = ['jitter', 'shimmer', 'rp', 'f0', 'energy']
    # for feat in allfeats:
    featname = 'f0'

    metadata = load_feat(base_folder_path, feature_name=featname)


In [ ]:
import pandas as pd
from pymer4.models import Lmer

from load_feat_pd import *


def lme(metadata, featname='shimmer', level='utt', stats='mean'):

    df = pd.DataFrame(metadata)
    
    # remove group id == '11':
    df = df[df['group_id'] != '11']
    
    # map group_id 21 to HC 22 to PD
    df['group_id'] = df['group_id'].map({'21': 'HC', '22': 'PD'})


    # change age from string to int
    df['age'] = pd.to_numeric(df['age'], errors='coerce')
    missing_age = df['age'].isnull().sum()
    if missing_age > 0:
        print(f"warning 'age' has {missing_age} NaN, has been deleted")
        df = df.dropna(subset=['age'])
        
    # add one more column item name in the filename
    df['item'] = df['filename'].apply(lambda x: x.split('.')[0].split('_')[-1])



    if level == 'utt':
        # change the value from np array to float value
        df['value'] = df['value'].apply(lambda x: x[0])
        
    elif level == 'frame':
        if stats == 'mean':
            df['value'] = df['value'].apply(lambda x: x.mean())
        elif stats == 'std':
            df['value'] = df['value'].apply(lambda x: x.std())
        elif stats == 'median':
            df['value'] = df['value'].apply(lambda x: np.median(x))

    # print(df.head())
    # print(df.shape)
    featname = "response_time" if featname == 'rp' else featname
    valuetype = stats if level == 'frame' else ''

    print(f'=============== value: {featname}_{valuetype} ====================\n')


    model = Lmer("value ~ age + group_id*experiment + group_id*gender + (1|subject_id) + (1|item)", data=df)
                        

    result = model.fit()

    with open('./res_v1.txt', 'a') as f:
        f.write(f'\n')
        f.write(f'=============== value: {featname}_{valuetype} ====================\n')
        f.write(result[['Estimate', 'SE', 'T-stat', 'P-val', 'Sig']].to_string())    
        f.write(f'\n')
        f.write(model.ranef_var.to_string())
        f.write(f'\n')


        
if __name__ == '__main__':
    base_folder_path_unnorm = Path('/data/storage025/Turntaking/wavs_single_channel_nosil')
    base_folder_path = Path('/data/storage025/Turntaking/wavs_single_channel_normalized_nosil')

    feats2level = {
        'jitter': 'utt',
        'shimmer': 'utt',
        'rp': 'utt',
        'f0': 'frame',
        'energy': 'frame'
    }

    np.set_printoptions(precision=2)
    allfeats = ['jitter', 'shimmer', 'rp', 'f0', 'energy']
    # for feat in allfeats:
    
    for featname in allfeats:
        folder = base_folder_path_unnorm if featname == 'energy' else base_folder_path

        if feats2level[featname] == 'frame':
            metadata = load_feat(folder, feature_name=featname)

            lme(metadata, featname=featname, level=feats2level[featname], stats='mean')
            lme(metadata, featname=featname, level=feats2level[featname], stats='std')
            lme(metadata, featname=featname, level=feats2level[featname], stats='median')
        else:
            metadata = load_feat(folder, feature_name=featname)
            lme(metadata, featname=featname, level=feats2level[featname])


